In [16]:
import requests
import datetime

api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.96&longitude=7.63&hourly=temperature_2m,rain,snowfall,windspeed_10m&daily=weathercode&timezone=Europe%2FBerlin&start_date={start}&end_date={end}"

def get_score(day: datetime.datetime) -> float:
    formatted = day.strftime("%Y-%m-%d")
    res = requests.get(api_url.format(start=formatted, end=formatted)).json()
    hourly = res["hourly"]

    temp = hourly["temperature_2m"]
    rain = hourly["rain"]
    snowfall = hourly["snowfall"]
    wind = hourly["windspeed_10m"]

    hour = day.hour + (1 if day.minute >= 20 else 0)

    c_temp = temp[hour]
    c_rain = rain[hour]
    c_snow = snowfall[hour]
    c_wind = wind[hour]

    print("c_temp:", c_temp, "| c_rain:", c_rain, "| c_snow:", c_snow, "| c_wind:", c_wind)

    points = 0
    max_points = 0

    max_points += 15
    # in °C
    if c_temp < -5:
        points += 15
    elif c_temp < 0:
        points += 5
    elif c_temp < 5:
        points += 2
    elif c_temp < 10:
        points += 1

    max_points += 15
    # in mm
    if c_rain >= 30:
        points += 15
    elif c_rain >= 10:
        points += 8
    elif c_rain >= 2.5:
        points += 4
    elif c_rain >= 0:
        points += 1

    max_points += 15
    # in cm
    if c_snow >= 10:
        points += 15
    elif c_snow >= 3:
        points += 4
    elif c_snow >= 1:
        points += 2
    elif c_snow >= 0:
        points += 1

    max_points += 15
    # in km/h
    if c_wind >= 60: # stormy wind
        points += 15
    elif c_wind >= 40: # strong wind
        points += 9
    elif c_wind >= 30: # fresh breeze
        points += 3
    elif c_wind >= 12: # faint breeze
        points += 1

    print(points, points ** 1.3, max_points)
    return 1 - min(points ** 1.3 / max_points, 1.0)

get_score(datetime.datetime.now())

c_temp: 8.1 | c_rain: 0.1 | c_snow: 0.0 | c_wind: 10.4
3 4.171167510947728 60


0.9304805414842046